In [16]:
import json
import numpy as np
import os
import chardet
from pathlib import Path

# ===== 配置区域 =====
dataset_path = Path("/root/autodl-tmp/ngp-workspace/data/mydata")
colmap_output = dataset_path / "sparse/0"
images_dir = dataset_path / "images"
# ===================

try:
    print(f"正在读取 COLMAP 输出: {colmap_output}/images.txt...")
    
    # 1. 智能检测文件编码
    encoding_detect = "mbcs"  # 默认值
    try:
        with open(colmap_output / "images.txt", "rb") as f:
            raw_data = f.read(4096)
            result = chardet.detect(raw_data)
            encoding_detect = result["encoding"] or "mbcs"
            print(f"检测到文件编码: {encoding_detect}")
    except Exception as e:
        print(f"编码检测失败，使用默认编码: {str(e)}")
    
    # 2. 读取文件（使用检测到的编码）
    with open(colmap_output / "images.txt", "r", encoding=encoding_detect, errors="replace") as f:
        lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]
    
    # 3. 解析相机参数（每2行为一组数据）
    frames = []
    for i in range(0, len(lines), 2):
        parts = lines[i].split()
        
        # 确认数据完整性
        if len(parts) < 10:
            continue
            
        # 提取文件名（最后部分）
        image_name = " ".join(parts[9:])
        qvec = np.array(parts[1:5], dtype=float)  # 四元数
        tvec = np.array(parts[5:8], dtype=float)  # 平移
        
        # 四元数转旋转矩阵
        q0, q1, q2, q3 = qvec
        R = np.array([
            [1 - 2*(q2**2 + q3**2), 2*(q1*q2 - q0*q3),     2*(q1*q3 + q0*q2)],
            [2*(q1*q2 + q0*q3),     1 - 2*(q1**2 + q3**2), 2*(q2*q3 - q0*q1)],
            [2*(q1*q3 - q0*q2),     2*(q2*q3 + q0*q1),     1 - 2*(q1**2 + q2**2)]
        ])
        
        # 构建4x4变换矩阵
        transform = np.eye(4)
        transform[:3, :3] = R
        transform[:3, 3] = tvec
        
        # 添加到帧列表
        frames.append({
            "file_path": f"./images/{image_name}", 
            "transform_matrix": transform.tolist()
        })
    
    # 4. 生成transforms.json
    print(f"成功解析 {len(frames)} 个相机位姿")
    camera_angle_x = 0.85  # 简化处理
    
    output_file = dataset_path / "transforms.json"
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump({
            "camera_angle_x": camera_angle_x,
            "frames": frames
        }, f, indent=4, ensure_ascii=False)  # ensure_ascii=False保留非ASCII字符
    
    print(f"✅ 转换成功！配置文件保存至: {output_file}")
    print(f"✅ 处理了 {len(frames)} 张照片的位姿数据")
    
    # 额外验证：显示前3个文件
    print("\n==== 示例数据 ====")
    for i, frame in enumerate(frames[:3]):
        print(f"{i+1}. {frame['file_path']}")

except Exception as e:
    print(f"❌ 转换失败: {str(e)}")
    import traceback
    traceback.print_exc()  # 打印完整错误堆栈
    
    print("\n请检查以下内容:")
    print(f"1. 文件路径: {colmap_output}/images.txt")
    print(f"2. 文件是否存在: {'✅存在' if (colmap_output/'images.txt').exists() else '❌不存在'}")
    print(f"3. 文件大小: {os.path.getsize(colmap_output/'images.txt') if os.path.exists(colmap_output/'images.txt') else 0} 字节")

正在读取 COLMAP 输出: /root/autodl-tmp/ngp-workspace/data/mydata/sparse/0/images.txt...
检测到文件编码: ISO-8859-1
成功解析 53 个相机位姿
✅ 转换成功！配置文件保存至: /root/autodl-tmp/ngp-workspace/data/mydata/transforms.json
✅ 处理了 53 张照片的位姿数据

==== 示例数据 ====
1. ./images/Î¢ÐÅÍ¼Æ¬_20250623142332.jpg
2. ./images/Î¢ÐÅÍ¼Æ¬_20250623142334.jpg
3. ./images/Î¢ÐÅÍ¼Æ¬_202506231423341.jpg


In [17]:
import json
from pathlib import Path

# 读取真实文件
path = Path("/root/autodl-tmp/ngp-workspace/data/mydata/transforms.json")
with open(path, 'r') as f:
    data = json.load(f)

print(f"帧数量: {len(data['frames'])}")
print("前三帧文件:")
for frame in data['frames'][:3]:
    print(f" - {frame['file_path']}")

帧数量: 53
前三帧文件:
 - ./images/Î¢ÐÅÍ¼Æ¬_20250623142332.jpg
 - ./images/Î¢ÐÅÍ¼Æ¬_20250623142334.jpg
 - ./images/Î¢ÐÅÍ¼Æ¬_202506231423341.jpg


In [19]:
import os
import json
import shutil
from pathlib import Path
import re

# ===== 配置区域 =====
base_path = Path("/root/autodl-tmp/ngp-workspace/data/mydata")  # 数据集根目录
# ===================

def remove_chinese(text):
    """去除字符串中的中文字符"""
    return re.sub(r'[\u4e00-\u9fff]', '', text)

print("开始清理所有中文名称...")

# 1. 清理所有图片文件
image_files = []
for ext in ["*.jpg", "*.JPG", "*.jpeg", "*.png", "*.PNG"]:
    image_files.extend(base_path.glob(f"images/{ext}"))

for img in image_files:
    new_name = remove_chinese(img.name)
    if new_name != img.name:
        new_path = img.with_name(new_name)
        img.rename(new_path)
        print(f"图片重命名: {img.name} → {new_name}")
    else:
        print(f"图片无需重命名: {img.name}")

# 2. 清理 transforms.json 中的中文路径
json_file = base_path / "transforms.json"
if json_file.exists():
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    for frame in data["frames"]:
        # 去除路径中的中文
        original_path = frame["file_path"]
        clean_path = remove_chinese(original_path)
        
        # 确保图片文件名与实际文件名匹配
        img_name = Path(clean_path).name
        actual_img = next(base_path.glob(f"images/{img_name}"), None)
        
        if actual_img:
            frame["file_path"] = clean_path
            print(f"更新文件路径: {original_path} → {clean_path}")
    
    # 保存清理后的配置文件
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
    print(f"✅ 已清理 transforms.json 中的中文路径")
else:
    print("❌ 未找到 transforms.json 文件")

# 3. 清理目录名称
for folder in base_path.iterdir():
    if folder.is_dir():
        clean_name = remove_chinese(folder.name)
        if clean_name != folder.name:
            new_folder = folder.with_name(clean_name)
            folder.rename(new_folder)
            print(f"目录重命名: {folder.name} → {clean_name}")

print("\n" + "="*50)
print("✅ 中文名称清理完成!")
print("="*50)

开始清理所有中文名称...
图片重命名: 微信图片_20250623142332.jpg → _20250623142332.jpg
图片重命名: 微信图片_20250623142334.jpg → _20250623142334.jpg
图片重命名: 微信图片_202506231423341.jpg → _202506231423341.jpg
图片重命名: 微信图片_20250623142335.jpg → _20250623142335.jpg
图片重命名: 微信图片_202506231423351.jpg → _202506231423351.jpg
图片重命名: 微信图片_202506231423352.jpg → _202506231423352.jpg
图片重命名: 微信图片_20250623142336.jpg → _20250623142336.jpg
图片重命名: 微信图片_202506231423361.jpg → _202506231423361.jpg
图片重命名: 微信图片_202506231423362.jpg → _202506231423362.jpg
图片重命名: 微信图片_202506231423363.jpg → _202506231423363.jpg
图片重命名: 微信图片_20250623142337.jpg → _20250623142337.jpg
图片重命名: 微信图片_202506231423371.jpg → _202506231423371.jpg
图片重命名: 微信图片_202506231423372.jpg → _202506231423372.jpg
图片重命名: 微信图片_202506231423373.jpg → _202506231423373.jpg
图片重命名: 微信图片_202506231423374.jpg → _202506231423374.jpg
图片重命名: 微信图片_20250623142338.jpg → _20250623142338.jpg
图片重命名: 微信图片_202506231423381.jpg → _202506231423381.jpg
图片重命名: 微信图片_202506231423382.jpg → _202506231423382.jpg
图片重命名: 微

In [20]:
import os
from pathlib import Path

# 验证所有图片名称
base_path = Path("/root/autodl-tmp/ngp-workspace/data/mydata")
image_dir = base_path / "images"

print("验证所有图片名称:")
for img in image_dir.glob("*"):
    # 检查是否包含中文
    if any('\u4e00' <= char <= '\u9fff' for char in img.name):
        print(f"❌ 检测到中文: {img.name}")
    else:
        print(f"✅ 通过: {img.name}")

# 验证 transforms.json
json_file = base_path / "transforms.json"
if json_file.exists():
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    print("\n验证 transforms.json:")
    for i, frame in enumerate(data["frames"][:5]):  # 检查前5个示例
        path = frame["file_path"]
        if any('\u4e00' <= char <= '\u9fff' for char in path):
            print(f"❌ 检测到中文路径: {path}")
        else:
            print(f"✅ 通过: {path}")
else:
    print("❌ 未找到 transforms.json")

验证所有图片名称:
✅ 通过: _20250623142334.jpg
✅ 通过: _202506231423341.jpg
✅ 通过: _20250623142335.jpg
✅ 通过: _202506231423351.jpg
✅ 通过: _202506231423352.jpg
✅ 通过: _20250623142336.jpg
✅ 通过: _202506231423361.jpg
✅ 通过: _202506231423362.jpg
✅ 通过: _202506231423363.jpg
✅ 通过: _20250623142337.jpg
✅ 通过: _202506231423371.jpg
✅ 通过: _202506231423372.jpg
✅ 通过: _202506231423373.jpg
✅ 通过: _202506231423374.jpg
✅ 通过: _20250623142338.jpg
✅ 通过: _202506231423381.jpg
✅ 通过: _202506231423382.jpg
✅ 通过: _202506231423383.jpg
✅ 通过: _202506231423384.jpg
✅ 通过: _20250623142339.jpg
✅ 通过: _202506231423391.jpg
✅ 通过: _202506231423392.jpg
✅ 通过: _202506231423393.jpg
✅ 通过: _20250623142340.jpg
✅ 通过: _202506231423401.jpg
✅ 通过: _202506231423402.jpg
✅ 通过: _202506231423403.jpg
✅ 通过: _20250623142341.jpg
✅ 通过: _202506231423411.jpg
✅ 通过: _202506231423412.jpg
✅ 通过: _202506231423413.jpg
✅ 通过: _20250623142342.jpg
✅ 通过: _202506231423421.jpg
✅ 通过: _202506231423422.jpg
✅ 通过: _20250623142343.jpg
✅ 通过: _202506231423431.jpg
✅ 通过: _202506231423432.jpg
✅